# NLP with Disaster Tweets

```
Step 1. Library Import & Data Load
Step 2. Data Preprocessing
     2-a. Drop Columns
     2-b. Tokenizer
     2-c. Pad Sequences
     2-d. Match Data type to numpy.ndarray
Step 3. Modeling
Step 4. Model Compile
Step 5. Callbacks
Step 6. Model Fit
Step 7. Model Evaluate & Save
Step 8. Reload Model
Step 9. Predict Test Data
```


## Step 1. Library Import & Data Load

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
train_df = pd.read_csv('./data/train_data.csv')
test_df = pd.read_csv('./data/test_data.csv')

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this # earthquake...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask . Canada,1
2,5,NaN,NaN,All residents asked to ' shelter in place ' ...,1
3,6,NaN,NaN,"13,000 people receive # wildfires evacuation ...",1
4,7,NaN,NaN,Just got sent this photo from Ruby # Alaska a...,1


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [13]:
train_df.nunique()

id          7613
keyword      221
location    3341
text        6981
target         2
dtype: int64

In [16]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about # earthquake is different cities,..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting . # Spokane # wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [17]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [19]:
test_df.nunique()

id          3263
keyword      221
location    1602
text        3109
dtype: int64

## Step 2. Data Preprocessing

### 2-a. Drop Columns

In [21]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this # earthquake...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask . Canada,1
2,5,NaN,NaN,All residents asked to ' shelter in place ' ...,1
3,6,NaN,NaN,"13,000 people receive # wildfires evacuation ...",1
4,7,NaN,NaN,Just got sent this photo from Ruby # Alaska a...,1


In [22]:
train_df.drop(columns=['id','keyword','location'], axis=1, inplace=True)

In [23]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about # earthquake is different cities,..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting . # Spokane # wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [25]:
test_df.drop(columns=['id','keyword','location'],axis=1, inplace=True)

In [26]:
print(train_df.shape, test_df.shape)

(7613, 2) (3263, 1)


### 2-b. Tokenizer

In [34]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df['text'],train_df['target'], test_size=0.2, random_state=111)

In [43]:
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)

(6090,) (6090,) (1523,) (1523,)


In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [44]:
vocab_size = 1000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)

In [45]:
tokenizer.fit_on_texts(X_train)

In [46]:
X_train = tokenizer.texts_to_sequences(X_train)
X_valid = tokenizer.texts_to_sequences(X_valid)

In [55]:
for i in range(10):
    print(len(X_train[i]))

11
16
10
28
16
20
15
20
10
8


In [61]:
X_train[0]

[132, 1, 6, 253, 10, 15, 685, 12, 618, 157, 1]

In [56]:
for i in range(10):
    print(len(X_valid[i]))

11
26
27
21
12
7
12
7
7
11


In [62]:
X_valid[0]

[32, 11, 230, 494, 18, 758, 402, 923, 178, 2, 1]

### 2-c. Pad Sequences

In [64]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [66]:
max_length = 120
trunc_type = 'post'
pad_type = 'post'

In [67]:
X_train_padded = pad_sequences(X_train, maxlen=max_length, truncating=trunc_type, padding=pad_type)
X_valid_padded = pad_sequences(X_valid, maxlen=max_length, truncating=trunc_type, padding=pad_type)

In [72]:
X_train_padded[:2]

array([[132,   1,   6, 253,  10,  15, 685,  12, 618, 157,   1,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [821,   1, 138,  26,   1,   5,   3,   1,  25, 125,   1,  70, 174,
         13,   1,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,

In [73]:
X_valid_padded[:2]

array([[ 32,  11, 230, 494,  18, 758, 402, 923, 178,   2,   1,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 17,   1,  50, 558,  57,   5, 834,   3,   1,   1,   9,  80,  54,
        437,  36,  20,   3,   1,   9,  54,   1,  12,   1,  33,   1,   1,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,

In [74]:
print(X_train_padded.shape, X_valid_padded.shape)

(6090, 120) (1523, 120)


### 2-d. Match Data type to numpy.ndarray

In [80]:
print(type(X_train_padded), type(X_valid_padded))
print(type(y_train), type(y_valid))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [81]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)

In [82]:
print(type(X_train_padded), type(X_valid_padded))
print(type(y_train), type(y_valid))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


## Step 3. Modeling

In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten

In [83]:
embedding_dim = 16
# vocab_size = 1000
# max_length = 120

In [87]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64, dropout=0.5)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [88]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 128)          41472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120, 128)          98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1

## Step 4. Model Compile

In [89]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

## Step 5. Callbacks

In [86]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [90]:
filepath = 'my_checkpoint.ckpt'
cp = ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

In [91]:
ep = EarlyStopping(
    monitor='val_loss', 
    patience=5,
)

## Step 6. Model Fit

In [92]:
epochs=30
model.fit(
    X_train_padded, y_train,
    validation_data = (X_valid_padded, y_valid),
    callbacks=[cp,ep],
    epochs=epochs
)

Epoch 1/30
191/191 [==============================] - ETA: 0s - loss: 0.5619 - acc: 0.6980
Epoch 00001: val_loss improved from inf to 0.47658, saving model to my_checkpoint.ckpt
191/191 [==============================] - 37s 193ms/step - loss: 0.5619 - acc: 0.6980 - val_loss: 0.4766 - val_acc: 0.7735
Epoch 2/30
191/191 [==============================] - ETA: 0s - loss: 0.4421 - acc: 0.8064
Epoch 00002: val_loss improved from 0.47658 to 0.44776, saving model to my_checkpoint.ckpt
191/191 [==============================] - 31s 161ms/step - loss: 0.4421 - acc: 0.8064 - val_loss: 0.4478 - val_acc: 0.7945
Epoch 3/30
191/191 [==============================] - ETA: 0s - loss: 0.4026 - acc: 0.8297
Epoch 00003: val_loss improved from 0.44776 to 0.43836, saving model to my_checkpoint.ckpt
191/191 [==============================] - 30s 159ms/step - loss: 0.4026 - acc: 0.8297 - val_loss: 0.4384 - val_acc: 0.7984
Epoch 4/30
191/191 [==============================] - ETA: 0s - loss: 0.3882 - acc: 0.

## Step 7. Model Evaluate & Save

In [93]:
model.load_weights(filepath)

In [94]:
model.evaluate(X_valid_padded, y_valid)

48/48 [==============================] - 2s 32ms/step - loss: 0.4384 - acc: 0.7984


[0.4383638799190521, 0.7984241843223572]

In [97]:
X_valid[0]

[32, 11, 230, 494, 18, 758, 402, 923, 178, 2, 1]

In [107]:
model.save('./model/basic_val_loss_0.4383.h5')

## Step 8. Reload Model

In [110]:
import tensorflow as tf

In [111]:
mymodel = tf.keras.models.load_model('./model/basic_val_loss_0.4383.h5')

In [112]:
mymodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 128)          41472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120, 128)          98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1

## Step 9. Predict Test Data 

In [117]:
X_test = tokenizer.texts_to_sequences(test_df['text'])

In [120]:
X_test_padded = pad_sequences(X_test, maxlen=max_length, truncating=trunc_type, padding=pad_type)

In [123]:
y_test_raw = model.predict(X_test_padded)

In [124]:
y_test_raw

array([[0.69282794],
       [0.5737326 ],
       [0.68215466],
       ...,
       [0.96064204],
       [0.9371407 ],
       [0.9780343 ]], dtype=float32)

In [131]:
y_test = list(map(lambda x : 1 if x > 0.5 else 0, y_test_raw))

In [144]:
set(y_test)

{0, 1}

In [134]:
y_test[:5]

[1, 1, 1, 0, 1]

In [147]:
test_df['predict'] = y_test

In [148]:
test_df

,text,predict
0,Just happened a terrible car crash,1
1,"Heard about # earthquake is different cities,...",1
2,"there is a forest fire at spot pond, geese are...",1
3,Apocalypse lighting . # Spokane # wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES SAFETY FASTENER...,1
3259,Storm in RI worse than last hurricane . My ci...,1
3260,Green Line derailment in Chicago,1
3261,MEG issues Hazardous Weather Outlook ( HWO ),1


In [150]:
test_df[test_df['predict']==1]

,text,predict
0,Just happened a terrible car crash,1
1,"Heard about # earthquake is different cities,...",1
2,"there is a forest fire at spot pond, geese are...",1
4,Typhoon Soudelor kills 28 in China and Taiwan,1
5,We are shaking . It is an earthquake,1
...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES SAFETY FASTENER...,1
3259,Storm in RI worse than last hurricane . My ci...,1
3260,Green Line derailment in Chicago,1
3261,MEG issues Hazardous Weather Outlook ( HWO ),1


In [149]:
test_df[test_df['predict']==0]

,text,predict
3,Apocalypse lighting . # Spokane # wildfires,0
6,They ' d probably still show more life than Ar...,0
7,Hey ! How are you ?,0
8,What a nice hat ?,0
9,Fuck off !,0
...,...,...
3247,RT CNBC ' 3 words from Disney CEO Bob Iger wr...,0
3248,Smackdown tyme this should put me in a good mo...,0
3249,@ thrillhho jsyk I have not stopped thinking ...,0
3250,@ Sam Tighe Begovic has been garbage . He go...,0
